In [14]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn import datasets
from sklearn import metrics
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import

In [15]:
mnist = datasets.fetch_openml('mnist_784')
X = np.array(mnist.data.astype('float32'))
y = np.array(mnist.target.astype('int'))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the pixel values to the range [0, 1]
X_train /= 255.0
X_test /= 255.0

/home/prat/miniconda3/envs/rp/lib/python3.12/site-packages/sklearn/datasets/_openml.py:1022: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [16]:
def train_sgd(X_train, X_test, y_train, y_test):
    # Create an SGDClassifier with a pipeline for scaling and classification
    sgd_classifier = make_pipeline(StandardScaler(), SGDClassifier(max_iter=500, tol=1e-3))

    # Train the classifier on the training data
    sgd_classifier.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = sgd_classifier.predict(X_test)

    # Evaluate the model
    accuracy = metrics.accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy:.2%}")

    # Print a classification report
    print(metrics.classification_report(y_test, y_pred))
    return accuracy

In [17]:
from sklearn.utils import shuffle

# Separate data by classes in the training set
classes = np.unique(y_train)
class_data = {cls: X_train[y_train == cls] for cls in classes}


def reduce_data(x_percent):
    # Randomly sample x% from each class
    sampled_data = {cls: class_data[cls][np.random.choice(class_data[cls].shape[0], int(x_percent * class_data[cls].shape[0]), replace=False)] for cls in classes}

    # Combine sampled data
    new_X_train = np.concatenate(list(sampled_data.values()), axis=0)
    new_y_train = np.concatenate([np.full_like(sampled_data[cls][:, 0], cls) for cls in classes])

    # Shuffle the new training set
    new_X_train, new_y_train = shuffle(new_X_train, new_y_train, random_state=42)

    return new_X_train, new_y_train



list_of_percentages = [i/10 for i in range(1, 11)]
list_of_train_y_set = [(x,y) for x,y in [reduce_data(i) for i in list_of_percentages]]

In [ ]:
print("Training SVM with different training set sizes...")
l =  [[train_sgd(X_train, X_test, y_train, y_test) for X_train, y_train in list_of_train_y_set] for i in range(10)]

Training SVM with different training set sizes...


/home/prat/miniconda3/envs/rp/lib/python3.12/site-packages/sklearn/linear_model/_stochastic_gradient.py:713: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


Accuracy: 89.22%
              precision    recall  f1-score   support

           0       0.95      0.94      0.95      1343
           1       0.96      0.95      0.95      1600
           2       0.88      0.89      0.88      1380
           3       0.86      0.86      0.86      1433
           4       0.90      0.89      0.89      1295
           5       0.83      0.86      0.85      1273
           6       0.94      0.94      0.94      1396
           7       0.92      0.89      0.90      1503
           8       0.85      0.83      0.84      1357
           9       0.83      0.86      0.85      1420

    accuracy                           0.89     14000
   macro avg       0.89      0.89      0.89     14000
weighted avg       0.89      0.89      0.89     14000



/home/prat/miniconda3/envs/rp/lib/python3.12/site-packages/sklearn/linear_model/_stochastic_gradient.py:713: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


Accuracy: 90.38%
              precision    recall  f1-score   support

           0       0.97      0.95      0.96      1343
           1       0.95      0.96      0.96      1600
           2       0.90      0.88      0.89      1380
           3       0.90      0.87      0.88      1433
           4       0.92      0.90      0.91      1295
           5       0.85      0.85      0.85      1273
           6       0.93      0.94      0.94      1396
           7       0.93      0.92      0.92      1503
           8       0.81      0.88      0.84      1357
           9       0.87      0.87      0.87      1420

    accuracy                           0.90     14000
   macro avg       0.90      0.90      0.90     14000
weighted avg       0.90      0.90      0.90     14000



In [ ]:
# Plot the results
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
for i in l:
    plt.scatter(list_of_percentages, i)
plt.show()
